In [1]:
import pickle
import pandas as pd
#Rawdata.pkl is created in "Makedata.py"
with open("Rawdata.pkl", "rb") as load:
    X = pickle.load(load)

In [2]:
# Data preprocessing
# Drop empty fields
print(len(X))
X = X.dropna()
print(len(X))

10815072
10815004


In [3]:
#Lowercase everything
X['Text'] = X['Text'].astype(str)
X['Text'] = X['Text'].apply(lambda text: text.lower())

In [4]:
# Pre-make train and test data, avoid repetitions
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
import numpy as np
X.reset_index(inplace=True, drop=True)

In [5]:
X_train, X_test, y_train_lat, y_test_lat = train_test_split(X['Text'], X['Lat'], test_size=0.1, random_state=100)
_, _, y_train_long, y_test_long = train_test_split(X['Text'], X['Long'], test_size=0.1, random_state=100)

vectorizer = TfidfVectorizer(analyzer='word',lowercase=False, min_df=100)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
print(X_train.shape)

(9733503, 51783)


In [6]:
from sklearn.decomposition import TruncatedSVD
reducer = TruncatedSVD(30)
X_train_trunc = reducer.fit_transform(X_train)
X_test_trunc = reducer.transform(X_test)

In [7]:
from lightgbm.sklearn import LGBMRegressor

model_lat = LGBMRegressor(n_jobs=-1, n_estimators=2500, learning_rate=0.35, num_leaves=9120)
model_lat.fit(X_train_trunc, y_train_lat)

model_long = LGBMRegressor(n_jobs=-1, n_estimators=2500, learning_rate=0.35, num_leaves=9120)
model_long.fit(X_train_trunc, y_train_long)

LGBMRegressor(learning_rate=0.35, n_estimators=2500, num_leaves=9120)

In [8]:

preds_lat = model_lat.predict(X_test_trunc)
preds_long = model_long.predict(X_test_trunc)

with open("LAT_PRED.pkl", "wb") as save:
    pickle.dump(preds_lat, save)

with open("LONG_PRED.pkl", "wb") as save:
    pickle.dump(preds_long, save)


NameError: name 'mean_absolute_error' is not defined

In [9]:
from sklearn.metrics import mean_absolute_error

print(f"LAT ERROR: {mean_absolute_error(y_test_lat, preds_lat)}")
print(f"LONG ERROR: {mean_absolute_error(y_test_long, preds_long)}")

LAT ERROR: 5.804093798198766
LONG ERROR: 3.6656025520347124


In [16]:
from geopy import distance

def get_km_error(lat_true, long_true, lat_pred, long_pred):
    return distance.distance((lat_true, long_true), (lat_pred, long_pred)).km

from statistics import mean
print(mean([get_km_error(lat_true, long_true, lat_pred, long_pred) for lat_true, long_true, lat_pred, long_pred in
           zip(y_test_lat, y_test_long, preds_lat, preds_long)]))

824.8049995035177
